<a href="https://colab.research.google.com/github/Alex-2911/Basketball-Projection/blob/main/_1_24112024_get_data_previous_game_day_and_parse_to_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
# !dpkg -i google-chrome-stable_current_amd64.deb
# !apt-get -f install -y

# !wget https://chromedriver.storage.googleapis.com/$(curl -sS https://chromedriver.storage.googleapis.com/LATEST_RELEASE)/chromedriver_linux64.zip
# !unzip chromedriver_linux64.zip
# !mv chromedriver /usr/bin/chromedriver
# !chmod +x /usr/bin/chromedriver


In [2]:
import time
import calendar
import datetime
from datetime import datetime, timedelta

import re
from bs4 import BeautifulSoup

current_season = 2025

#today = datetime.now()
today = datetime.now() - timedelta(days=0)
print(today)
yesterday = datetime.now() - timedelta(days=1)


import datetime
#today_str_format = datetime.datetime.now().strftime("%Y-%m-%d")
today_str_format = today.strftime("%Y-%m-%d")

2024-12-01 19:20:23.470806


In [3]:
# Get today's date
# Determine the current and next month/year

if today.day == 1:
    current_month = today.month - 1

    if current_month == 0:
        current_month = 12
        current_year = today.year - 1
        last_month = 12
        last_month_name = calendar.month_name[last_month].lower()
    else:
        current_year = today.year
        last_month = current_month# - 1
        last_month_name = calendar.month_name[last_month].lower()
        print(last_month)
else:
    current_month = today.month
    current_year = today.year
    last_month = None  # Not used in this case

11


In [4]:
# import os
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define base directory in Google Drive
#DATA_DIR = os.path.join("/content/drive/My Drive", "NBA_Script", "2025", "Gathering_Data", "data")
#SRC_DIR = os.path.join("/content/drive/My Drive", "NBA_Script", "2025", "Gathering_Data", "Whole_Statistic")

# Define directories
base_dir = "/content/drive/My Drive/NBA_Script/2025/Gathering_Data"
STANDINGS_DIR = os.path.join(base_dir, f"{current_season}_standings")
SCORES_DIR = os.path.join(base_dir, f"{current_season}_scores")
SRC_DIR = os.path.join(base_dir, f"Whole_Statistic")


# Define subdirectories
#STANDINGS_DIR = os.path.join(DATA_DIR, f"{current_season}_standings")
#SCORES_DIR = os.path.join(DATA_DIR, f"{current_season}_scores") #Fixed: Removed the extra 'S' and uncommented the line

# Create directories if they don't exist
os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

#print(f"Data Directory: {DATA_DIR}") #Fixed: DATA_DIR is commented out above and not defined, causing a NameError
print(f"Standings Directory: {STANDINGS_DIR}")
print(f"Scores Directory: {SCORES_DIR}")
print(f"Whole_Statistic Directory: {SRC_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Standings Directory: /content/drive/My Drive/NBA_Script/2025/Gathering_Data/2025_standings
Scores Directory: /content/drive/My Drive/NBA_Script/2025/Gathering_Data/2025_scores
Whole_Statistic Directory: /content/drive/My Drive/NBA_Script/2025/Gathering_Data/Whole_Statistic


In [5]:
def get_current_year_and_month():
    """
    Returns the current year and month.
    """
    today = datetime.now()
    return today.year, today.month

In [6]:
!pip install selenium webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, WebDriverException
import os
import logging
import time

# Configure logging
# Initialize chrome_options before using it
chrome_options = Options()
chrome_options.add_argument("--disable-ipv6")
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def get_html(url, selector, sleep=5, retries=3, headless=True):
    """
    Retrieves HTML content from a webpage using Selenium WebDriver.
    """
    html = None
    # Initialize driver to None outside the loop
    driver = None
    for attempt in range(retries):
        try:
            # WebDriver options
            # chrome_options is now defined globally
            if headless:
                chrome_options.add_argument("--headless")
                chrome_options.add_argument("--no-sandbox")
                chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource issues

            # Use ChromeDriverManager to manage the driver
            service = Service(ChromeDriverManager().install())
            driver = webdriver.Chrome(service=service, options=chrome_options)

            driver.get(url)
            time.sleep(sleep * (2 ** attempt))  # Exponential backoff
            element = driver.find_element(By.CSS_SELECTOR, selector)
            html = element.get_attribute("innerHTML")

            driver.quit() # Quit the driver after successful retrieval
            break # Exit the retry loop if successful

        except TimeoutException:
            logging.warning(f"Attempt {attempt + 1}: Timeout error on {url}. Retrying...")
            if driver is not None: # Now driver is defined before this line
              driver.quit() #Quit the driver before retrying
        except WebDriverException as e:
            logging.error(f"Webdriver error: {e}")
            if driver is not None: # Now driver is defined before this line
              driver.quit() #Quit the driver if any WebDriverException occurs
            break # Exit the loop if there's a WebDriverException

    if html is None:
        logging.error(f"Failed to retrieve HTML content from {url} after {retries} attempts.")

    return html

In [7]:
def scrape_season_for_month(season, month, month_name, standings_dir, get_html_function):
    """
    Scrapes NBA games data for a specific month and season from basketball-reference.com.

    Args:
        season (int): The NBA season year.
        month (int): The month number (1-12).
        month_name (str): The name of the month.
        standings_dir (str): Directory path to save the scraped data.
        get_html_function (function): Function to get HTML content from a URL.
    """
    #current_year, current_month = get_current_year_and_month()

    if season < current_year or (season == current_year and month < current_month):
        logging.warning("Invalid year or month already passed.")
        return

    logging.info(f"Scraping games for: {season}, {month_name.title()}")
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    selector = "#content .filter"
    html_content = get_html_function(url, selector)

    if not html_content:
        logging.error(f"Failed to retrieve data from {url}.")
        return

    soup = BeautifulSoup(html_content, 'html.parser')
    links = soup.find_all("a", href=re.compile("/leagues/NBA_[0-9]{4}_games-[a-z]+\.html"))
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]

    for url in standings_pages:
        save_path = os.path.join(standings_dir, url.split("/")[-1])
        if os.path.exists(save_path):
            logging.info(f"Path already exists: {save_path}")
            continue

        if month_name in save_path.lower():
            html = get_html_function(url, "#all_schedule")
            #print(html)
            if html:
                try:
                    with open(save_path, "w+", encoding='utf-8') as f:
                        f.write(html)
                    logging.info(f"Data for {month_name.title()} saved.")
                except Exception as e:
                    logging.error(f"Failed to save data for {month_name.title()}: {e}")
            else:
                logging.error(f"Failed to retrieve data for {month_name.title()} from {url}.")

# Usage example:
# season = 2024
# month = 1
# standings_dir = "path/to/standings_dir"
# scrape_monthly_games(season, month, standings_dir, get_html)


In [ ]:
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
# !pip install selenium
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.common.exceptions import TimeoutException, WebDriverException

In [ ]:
# def initialize_driver():
#     chrome_options = Options()
#     chrome_options.add_argument("--headless")
#     chrome_options.add_argument("--no-sandbox")
#     chrome_options.add_argument("--disable-dev-shm-usage")
#     service = Service(ChromeDriverManager().install())
#     return webdriver.Chrome(service=service, options=chrome_options)


In [8]:
import os
import calendar
import logging
from datetime import datetime

#chromedriver_path = r"D:\1. Python\3. chromedriver-win64\chromedriver-win64\chromedriver.exe"


# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize variables
current_season = 2025
today = datetime.now()
current_month = today.month
current_year = today.year

# Calculate next and last months
next_month = current_month + 1 if current_month < 12 else 1
next_year = current_year if next_month != 1 else current_year + 1
last_month = current_month - 1 if current_month > 1 else 12
last_year = current_year if last_month != 12 else current_year - 1

# Month names
current_month_name = calendar.month_name[current_month].lower()
next_month_name = calendar.month_name[next_month].lower()
last_month_name = calendar.month_name[last_month].lower()

# Define directories
#base_dir = "/content/drive/My Drive/NBA_Script/2025/Gathering_Data"
#STANDINGS_DIR = os.path.join(base_dir, f"{current_season}_standings")
#SCORES_DIR = os.path.join(base_dir, f"{current_season}_scores")

# Create directories
os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

# File removal logic
# File removal logic
if today.day == 1:  # Only delete previous month's file on the 1st
    file_to_remove = f"NBA_{current_season}_games-{last_month_name}.html"  # Use last_month_name (november)
    file_path = os.path.join(STANDINGS_DIR, file_to_remove)

    try:
        if os.path.exists(file_path):
            os.remove(file_path)
            logging.info(f"File {file_to_remove} has been removed.")
        else:
            logging.info(f"File {file_to_remove} does not exist.")
    except Exception as e:
        logging.error(f"An error occurred while removing {file_to_remove}: {e}")

try:
    if os.path.exists(file_path):
        os.remove(file_path)
        logging.info(f"File {file_to_remove} has been removed.")
    else:
        logging.info(f"File {file_to_remove} does not exist.")
except Exception as e:
    logging.error(f"An error occurred while removing {file_to_remove}: {e}")

# WebDriver initialization
#driver = initialize_driver()

try:
    # Scrape games for the appropriate months
    if today.day == 1:
        scrape_season_for_month(current_season, last_month, last_month_name, STANDINGS_DIR, get_html)
        scrape_season_for_month(current_season, next_month, next_month_name, STANDINGS_DIR, get_html)
    else:
        scrape_season_for_month(current_season, current_month, current_month_name, STANDINGS_DIR, get_html)

    # Example of processing game data (uncomment when functions are defined)
    # standings_files = os.listdir(STANDINGS_DIR)
    # for f in standings_files:
    #     if str(current_year) in f:
    #         filepath = os.path.join(STANDINGS_DIR, f)
    #         scrape_game(filepath, SCORES_DIR, driver)

finally:
    #driver.quit()
    logging.info("Driver session closed.")


In [9]:
import datetime
from datetime import datetime, timedelta


def scrape_game(standings_file, scores_dir, get_html_function):
    """
    Scrapes box scores for NBA games from the provided standings file.

    Args:
        standings_file (str): Path to the file containing the standings data.
        scores_dir (str): Directory path to save the scraped box scores.
        get_html_function (function): Function to get HTML content from a URL.
    """
    # Calculate the date of yesterday
    #yesterday = datetime.now() - timedelta(days=1)
    yesterday_date = yesterday.strftime("%Y%m%d")  # Format: YYYYMMDD

    with open(standings_file, 'r',encoding='utf-8') as f:
        html = f.read()

    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    filtered_urls = [url for url in box_scores if yesterday_date in url]

    for url in filtered_urls:
        save_path = os.path.join(scores_dir, url.split("/")[-1])

        if os.path.exists(save_path):
            continue

        html = get_html_function(url, "#content")

        if not html:
            continue

        try:
            with open(save_path, "wb+") as f:
                f.write(html.encode("utf-8"))
            print(f"Box score saved: {save_path}")
        except Exception as e:
            print(f"Failed to save box score: {e}")


In [10]:
def process_standings_files(standings_dir, current_season):
    """
    Process standings files for a specific season.

    Args:
        standings_dir (str): Directory containing standings files.
        current_season (int): The current NBA season year.
    """
    standings_files = os.listdir(standings_dir)
    print(standings_files)

    # Filter files for the current season
    files = [s for s in standings_files if str(current_season) in s]

    for f in files:
        filepath = os.path.join(standings_dir, f)
        print(filepath)

        scrape_game(filepath, SCORES_DIR, get_html)  # Assuming scrape_game is implemented

# Call the function with your STANDINGS_DIR and current_season
process_standings_files(STANDINGS_DIR, current_season)


['NBA_2025_games-october.html', 'NBA_2025_games-january.html', 'NBA_2025_games-november.html']
/content/drive/My Drive/NBA_Script/2025/Gathering_Data/2025_standings/NBA_2025_games-october.html
/content/drive/My Drive/NBA_Script/2025/Gathering_Data/2025_standings/NBA_2025_games-january.html
/content/drive/My Drive/NBA_Script/2025/Gathering_Data/2025_standings/NBA_2025_games-november.html
Box score saved: /content/drive/My Drive/NBA_Script/2025/Gathering_Data/2025_scores/202411300CHO.html
Box score saved: /content/drive/My Drive/NBA_Script/2025/Gathering_Data/2025_scores/202411300DET.html
Box score saved: /content/drive/My Drive/NBA_Script/2025/Gathering_Data/2025_scores/202411300MIL.html
Box score saved: /content/drive/My Drive/NBA_Script/2025/Gathering_Data/2025_scores/202411300PHO.html
Box score saved: /content/drive/My Drive/NBA_Script/2025/Gathering_Data/2025_scores/202411300UTA.html


In [11]:
from bs4 import BeautifulSoup

def get_first_game_date(standings_file):
    """
    Extract the date of the first game day from the standings file.

    Args:
        standings_file (str): Path to the standings HTML file.

    Returns:
        str: The date of the first game, formatted as 'Day, Month Date, Year'.
    """
    with open(standings_file, 'r', encoding='utf-8') as f:
        html = f.read()

    soup = BeautifulSoup(html, 'html.parser')

    # Find the first game date in the standings table
    table = soup.find("table", {"id": "schedule"})

    if not table:
        print(f"No schedule table found in {standings_file}.")
        return None

    # Look for the first non-header row (actual game data)
    first_game_row = table.find_all("tr")[1]  # Skip the header row and take the first game row
    if first_game_row:
        game_date_tag = first_game_row.find("th", {"data-stat": "date_game"})
        if game_date_tag:
            return game_date_tag.text.strip()  # Returns the date of the first game

    return None

# Example usage
standings_file =  os.path.join(STANDINGS_DIR, 'NBA_2025_games-october.html')  # Replace with the actual path to your file

first_game_date_str = get_first_game_date(standings_file)
#if first_game_date_str:
#    print(f"The first game is scheduled on: {first_game_date_str}")
#else:
#    print("No game dates found in the file.")


In [12]:
import os
import pandas as pd
import datetime
import logging
from io import StringIO
import numpy as np

MAX_DAYS_BACK = 150  # Configurable range for searching files

# Constants
#SCORE_DIR = "data/2025_scores"
#SRC_DIR = r'D:\1. Python\1. NBA Script\2025\Gathering_Data\Whole_Statistic'
#DST_DIR = r'D:\_Laufwerk C\11. Sorare\NBA\2025\Gathering_Data\Whole_Statistic'

# Assume first_game_date is already defined in your code

first_game_date = datetime.datetime.strptime(first_game_date_str, "%a, %b %d, %Y").date()


# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Check if the current date is before the season start date
today = datetime.date.today()
if today < first_game_date:
    logging.info(f"Current date ({today}) is before the NBA season start date ({first_game_date}). Script will not run.")
    exit()

# If the script reaches here, it means the season has started, and we can proceed


# Function to check if file exists
def file_exists(date_str):
    """Check if a specific file exists based on the date string."""
    filename = f"nba_games_{date_str}.csv"
    return os.path.isfile(os.path.join(SRC_DIR, filename))

# Function to find the most recent file
def find_most_recent_file(max_days=MAX_DAYS_BACK):
    """Find the most recent file within a specified number of days."""
    days_back = 1
    while days_back <= max_days:
        most_recent_date = (today - datetime.timedelta(days=days_back)).strftime("%Y-%m-%d")
        if file_exists(most_recent_date):
            logging.info(f"The file for {most_recent_date} exists.")
            return most_recent_date
        else:
            days_back += 1

    logging.warning("No file found within the specified range.")
    return None

# Main script execution
try:
    most_recent_date = find_most_recent_file()

    if most_recent_date:
        # Proceed with processing the found file
        # Add your processing logic here
        logging.info(f"Processing file for {most_recent_date}")
    else:
        # Handle the case when no file is found
        logging.warning("No recent data available to process.")

except FileNotFoundError:
    logging.error("File or directory not found.")
except IOError:
    logging.error("Error accessing file.")
except Exception as e:
    logging.error(f"An unexpected error occurred: {e}")


# #########################################################################################################################
# # CHECK FIRST IF THE SEASON HAS BEGUN AND THE DATA FROM PREVIOUS GAME DAY IS ACTUALLY AVAILABLE#
# ##############################################################################################################

In [13]:
today = datetime.date.today()
if today < first_game_date:
    logging.info(f"Current date ({today}) is before the NBA season start date ({first_game_date}). Script will not run.")
    exit()

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants
#SRC_DIR = r'D:\1. Python\1. NBA Script\2025\Gathering_Data\Whole_Statistic'
#SCORE_DIR = r'path_to_score_data'  # Define the path to score data

# Functions
def parse_html(box_score):
    """Parse HTML content from a box score file."""
    try:
        with open(box_score, encoding='utf-8') as f:
            html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        [s.decompose() for s in soup.select("tr.over_header, tr.thead")]
        return soup
    except Exception as e:
        logging.error(f"Error parsing HTML for {box_score}: {e}")
        return None

def read_season_info(soup):
    """Extract season information from the soup object."""
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    return os.path.basename(hrefs[1]).split("_")[0]

def read_line_score(soup):
    """Read line score from the soup object."""
    line_score = pd.read_html(StringIO(str(soup)), attrs={'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    line_score = line_score[["team", "total"]]
    return line_score

def read_stats(soup, team, stat):
    """Read team statistics from the soup object."""
    df = pd.read_html(StringIO(str(soup)), attrs={'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    return df.apply(pd.to_numeric, errors="coerce")

def copy_missing_files(src_dir, dst_dir):
    """Copy missing files from source to destination directory."""
    src_files = set(os.listdir(src_dir))
    dst_files = set(os.listdir(dst_dir))
    diff = src_files - dst_files

    for file_name in diff:
        if not file_name.startswith('.') and not file_name.endswith('.ipynb'):
            shutil.copy2(os.path.join(src_dir, file_name), dst_dir)
            logging.info(f'File {file_name} copied successfully')

In [14]:
# Main Code
# Main Code
def process_nba_data():
    """Main function to process NBA data."""
    #last_file_date = (datetime.date.today() - datetime.timedelta(days=days_back)).strftime("%Y-%m-%d")
    last_file_date = most_recent_date
    print(last_file_date)

    filename = f"nba_games_{last_file_date}.csv"
###########################################################################################################
    try:
        existing_statistics = pd.read_csv(os.path.join(SRC_DIR, filename))#, index_col=0)
###########################################################################################################
    except FileNotFoundError:
        logging.error(f"File {filename} not found in {SRC_DIR}.")
        return

    base_cols = None
    games = []

    box_scores = [os.path.join(SCORES_DIR, f) for f in os.listdir(SCORES_DIR) if f.endswith(".html")]

    # Check if box_scores is empty and log if it is
    if not box_scores:
        logging.warning(f"No .html files found in {SCORES_DIR}. Check if the directory is correct and contains the necessary files.")

    # ... inside process_nba_data function ...
    for box_score in box_scores:
        try:
            date = pd.Timestamp(os.path.basename(box_score)[:8]).date()

            # Print the date being checked to help diagnose the issue
            print(f"Checking date: {date}, yesterday: {pd.Timestamp(yesterday).date()}")

            # Original condition:
            # if date < pd.Timestamp(yesterday).date():
            #     continue  # This line might be skipping all files

            # Modified condition to only process files from yesterday or later
            # Consider changing it if a different date range is required:
            if date != pd.Timestamp(yesterday).date():
                continue

            # ... rest of the loop ...

            # ... rest of the loop ...

            soup = parse_html(box_score)
            if soup is None:
                continue

            line_score = read_line_score(soup)
            teams = list(line_score["team"])
            summaries = []

            for team in teams:
                basic = read_stats(soup, team, "basic")
                advanced = read_stats(soup, team, "advanced")

                totals = pd.concat([basic.iloc[-1], advanced.iloc[-1]])
                totals.index = totals.index.str.lower()

                maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
                maxes.index = maxes.index.str.lower() + "_max"

                summary = pd.concat([totals, maxes])
                if base_cols is None:
                    base_cols = [b for b in summary.index.drop_duplicates(keep="first") if "bpm" not in b]
                summary = summary[base_cols]
                summaries.append(summary)

            summary = pd.concat(summaries, axis=1).T
            game = pd.concat([summary, line_score], axis=1)
            game["home"] = [0, 1]

            game_opp = game.iloc[::-1].reset_index()
            game_opp.columns += "_opp"

            full_game = pd.concat([game, game_opp], axis=1)

            full_game["season"] = read_season_info(soup)
            full_game["date"] = pd.Timestamp(os.path.basename(box_score)[:8])
            full_game["won"] = full_game["total"] > full_game["total_opp"]
            games.append(full_game)

            if len(games) % 100 == 0:
                logging.info(f"{len(games)} / {len(box_scores)}")

        except Exception as e:
            logging.error(f"Error processing {box_score}: {e}")

    games_df = pd.concat(games, ignore_index=True)
    print(games_df.head(1).to_string(index=False))

    # Ensure there are no duplicated columns in games_df
    games_df = rename_duplicated_columns(games_df)

    # Reindex games_df to match the columns of existing_statistics
    games_df_reindexed = games_df.reindex(columns=existing_statistics.columns)

    # Rename duplicated columns in games_df by appending a suffix
    duplicated_columns = games_df.columns[games_df.columns.duplicated()]

    for col in duplicated_columns:
        count = 0
        for i in range(len(games_df.columns)):
            if games_df.columns[i] == col:
                games_df.columns.values[i] = f"{col}_{count}"
                count += 1


    #print(existing_statistics.head(60).to_string(index=False))


    file_name = f"nba_games_{today}.csv"
    print(file_name)
    #games_df.to_csv("nba_games.csv")
    #print(games_df.head(6).to_string(index=False))

    #new_statistics = pd.read_csv("nba_games.csv", index_col=0)
    #print(new_statistics.head(60).to_string(index=False))

    # if combined_statistics.shape[1] == existing_statistics.shape[1]:
    #     print("Both DataFrames have the same number of columns.")
    # else:
    #     print(f"Different number of columns: combined_statistics has {combined_statistics.shape[1]}, existing_statistics has {existing_statistics.shape[1]}.")

    # Check and print duplicated columns for diagnosis
    print("Existing Statistics Duplicated Columns:", existing_statistics.columns[existing_statistics.columns.duplicated()])
    print("Games DF Duplicated Columns:", games_df.columns[games_df.columns.duplicated()])


    existing_statistics.reset_index(drop=True, inplace=True)

    #print(existing_statistics.tail(1).to_string(index=False))

    # Align games_df columns to match existing_statistics
    games_df = games_df.reindex(columns=existing_statistics.columns)
    combined_statistics = pd.concat([existing_statistics, games_df], ignore_index=True)

    print(existing_statistics.columns.duplicated().any())
    print(games_df.columns.duplicated().any())


    # if 'mp' in combined_statistics.columns:
    #     # Get the index of the 'mp' column
    #     mp_col_index = combined_statistics.columns.get_loc('mp')
    #     # Drop 'mp' and all columns after it
    #     combined_statistics.drop(combined_statistics.columns[mp_col_index:], axis=1, inplace=True)


    combined_statistics.to_csv(os.path.join(SRC_DIR, file_name), index=False)
    #os.remove('nba_games.csv')

    # Copy any missing files from SRC_DIR to DST_DIR
    #copy_missing_files(SRC_DIR, DST_DIR)

def rename_duplicated_columns(df):
    """Rename duplicated columns by appending a suffix"""
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df

if __name__ == "__main__":
    process_nba_data()


2024-11-30
Checking date: 2024-11-23, yesterday: 2024-11-30
Checking date: 2024-11-23, yesterday: 2024-11-30
Checking date: 2024-11-23, yesterday: 2024-11-30
Checking date: 2024-11-23, yesterday: 2024-11-30
Checking date: 2024-11-23, yesterday: 2024-11-30
Checking date: 2024-11-23, yesterday: 2024-11-30
Checking date: 2024-11-23, yesterday: 2024-11-30
Checking date: 2024-11-30, yesterday: 2024-11-30
Checking date: 2024-11-30, yesterday: 2024-11-30
Checking date: 2024-11-30, yesterday: 2024-11-30
Checking date: 2024-11-30, yesterday: 2024-11-30
Checking date: 2024-11-30, yesterday: 2024-11-30
   mp    mp   fg  fga   fg%   3p  3pa   3p%   ft  fta   ft%  orb  drb  trb  ast  stl  blk  tov   pf   pts  gmsc  +/-   ts%  efg%  3par   ftr  orb%  drb%  trb%  ast%  stl%  blk%  tov%  usg%  ortg  drtg  mp_max  mp_max  fg_max  fga_max  fg%_max  3p_max  3pa_max  3p%_max  ft_max  fta_max  ft%_max  orb_max  drb_max  trb_max  ast_max  stl_max  blk_max  tov_max  pf_max  pts_max  gmsc_max  +/-_max  ts%_ma